In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.mixture import GaussianMixture
from sklearn.cluster import DBSCAN
from sklearn.decomposition import PCA
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, confusion_matrix, roc_curve, auc
from sklearn.model_selection import train_test_split

# Assuming your dataset has different column names
data = pd.read_csv("UCI_Credit_Card.csv")

# Assuming 'TotalCharges' is not present in the dataset
# No need to impute or drop columns since the dataset is assumed to be complete

# Assuming 'Churn' is the target variable
le = LabelEncoder()
data['default.payment.next.month'] = le.fit_transform(data['default.payment.next.month'])

X = data.drop('default.payment.next.month', axis=1)
y = data['default.payment.next.month']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Using Gaussian Mixture Model (GMM)
gmm = GaussianMixture(n_components=2, random_state=42)
gmm.fit(X_train)
cluster_labels_gmm = np.argmax(gmm.predict_proba(X_test), axis=1)

# Metrics
accuracy_gmm = accuracy_score(y_test, cluster_labels_gmm)
conf_matrix_gmm = confusion_matrix(y_test, cluster_labels_gmm)

# ROC Curve
probs_gmm = gmm.predict_proba(X_test)[:, 1]
fpr_gmm, tpr_gmm, thresholds_gmm = roc_curve(y_test, probs_gmm)
roc_auc_gmm = auc(fpr_gmm, tpr_gmm)

# Visualize Confusion Matrix
sns.heatmap(conf_matrix_gmm, annot=True, fmt='d', cmap='Blues')
plt.title('Confusion Matrix - GMM')
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.show()

# Visualize ROC Curve
plt.plot(fpr_gmm, tpr_gmm, color='darkorange', lw=2, label='ROC curve (area = {:.2f})'.format(roc_auc_gmm))
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC Curve - GMM')
plt.legend(loc="lower right")
plt.show()

# Using Density-Based Model (DBSCAN) for illustrative purposes
pca = PCA(n_components=2)
X_pca = pca.fit_transform(X)

dbscan = DBSCAN(eps=3, min_samples=5)
cluster_labels_dbscan = dbscan.fit_predict(X_pca)

# Visualize the clusters in 2D
sns.scatterplot(x=X_pca[:, 0], y=X_pca[:, 1], hue=cluster_labels_dbscan, palette='viridis')
plt.title('DBSCAN Clustering in 2D')
plt.show()
